In [ ]:
!pip install fosforml
!pip install statsmodels

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [20]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [21]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [22]:
sf_df = my_session.sql("select * from {}".format(df))

In [23]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [24]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                      0
TENURE_MONTHS                    0
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
CITY                             0
DISTANCE                         0
DEGREE_CLEAN                     0
ETHNICITY                        0
MARITAL_STATUS                   0
ROLE                             0
COMPANY_NAME                     0
ORGANIZATION_TYPE                0
ORGANIZATION_OWNERSHIP           0
STATE                            0
COUNTRY                          0
GENDER                           0
WORK_LIFE_BALANCE                0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFA

In [25]:
Original_df = pandas_df.dropna()

In [26]:
print(Original_df.isnull().sum())

EMPLOYEE_ID                  0
TENURE_MONTHS                0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
CITY                         0
DISTANCE                     0
DEGREE_CLEAN                 0
ETHNICITY                    0
MARITAL_STATUS               0
ROLE                         0
COMPANY_NAME                 0
ORGANIZATION_TYPE            0
ORGANIZATION_OWNERSHIP       0
STATE                        0
COUNTRY                      0
GENDER                       0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
dtype: int64


In [ ]:
# Ensure date columns are parsed correctly
date_columns = ['SCHOOL_ENDDATE', 'JOB_STARTDATE', 'JOB_ENDDATE']
for col in date_columns:
    Original_df[col] = pd.to_datetime(Original_df[col], errors='coerce')

In [ ]:
# Display the cleaned DataFrame
print(Original_df.head())

In [ ]:
# Assuming 'CHURN' is your target variable and 'TENURE_MONTHS' is a feature
y = Original_df['CHURN']
X = Original_df[['TENURE_MONTHS', 'AGE', 'SENIORITY']]  # Add more features as needed

In [ ]:
# Add a constant to the features
X = sm.add_constant(X)

In [ ]:
# Split data into training and testing sets
train_size = int(len(y) * 0.8)
y_train, y_test = y[:train_size], y[train_size:]
X_train, X_test = X[:train_size], X[train_size:]

In [ ]:
# Fit ARIMAX model
model = sm.tsa.ARIMA(y_train, exog=X_train, order=(1, 1, 1))
model_fit = model.fit()

In [27]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [28]:
df

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,JOB_ENDDATE,CHURN
0,<2 miles,Bachelors Degree,Accounting Manager,Horizon Financials,Financial Services,Private,2019-08-01,1
1,<2 miles,Undergraduate Diploma,Compliance Manager,Vertex Dynamics,Aerospace,Private,2021-06-01,1
3,<2 miles,Bachelors Degree,Process Assistant,Horizon Financials,Financial Services,Private,2017-07-01,1
4,<2 miles,Bachelors Degree,Operations Manager,BrightStart Ventures,E-commerce,Private,2019-08-01,1
6,>10 miles,Other,Operations Manager,BrightStart Ventures,E-commerce,Private,2018-07-01,1
...,...,...,...,...,...,...,...,...
299995,>10 miles,Masters Degree,Corporate Counsel,BridgePoint Capital,Investment,Public,2023-05-01,1
299996,>10 miles,Masters Degree,Compliance Manager,SecureTech Solutions,Technology,Public,2023-05-01,1
299997,>10 miles,Masters Degree,Process Assistant,GreenWorld Foundation,Environmental,Non-Profit,2023-05-01,1
299998,>10 miles,Masters Degree,Data Scientist,Pinnacle Technologies,Technology,Public,2023-05-01,1


In [29]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])

In [30]:
df.set_index('JOB_ENDDATE', inplace = True)

In [31]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]

In [32]:
# Group by categorical columns and resample monthly, then sum CHURN
New_df = df.groupby(["DISTANCE", "DEGREE_CLEAN", "ROLE", "COMPANY_NAME", "ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP"]).resample('M').agg({'CHURN': 'sum'}).reset_index()

In [33]:
# Filter rows where CHURN is greater than 0
df1 = New_df[New_df['CHURN'] > 0]

In [34]:
df1

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,JOB_ENDDATE,CHURN
0,2-5 miles,Associates Degree,Account Executive,Horizon Financials,Financial Services,Private,2021-10-31,1
1,2-5 miles,Associates Degree,IT Support Engineer,StateGrid Utilities,Utilities,State-Owned,2021-11-30,1
2,2-5 miles,Bachelors Degree,Accounting Manager,MetroHealth Systems,Healthcare,Non-Profit,2014-04-30,1
3,2-5 miles,Bachelors Degree,Accounting Manager,NextGen Innovations,Technology,Public,2019-06-30,1
4,2-5 miles,Bachelors Degree,Employee Relations Manager,BrightStart Ventures,E-commerce,Private,2022-07-31,1
...,...,...,...,...,...,...,...,...
290,>10 miles,Professional Certificate,Operations Manager,BlueSky Energy,Energy,Private,2010-11-30,1
291,>10 miles,Professional Certificate,Recruiter,MetroHealth Systems,Healthcare,Non-Profit,2017-04-30,1
292,>10 miles,Professional Certificate,Research Scientist,BrightStart Ventures,E-commerce,Private,2015-01-31,1
293,>10 miles,Professional Certificate,Solutions Architect,StateGrid Utilities,Utilities,State-Owned,2014-02-28,1


In [35]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [36]:
# Ensure the index is unique
df1 = df1[~df1.index.duplicated(keep='first')]

In [37]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

KeyError: Timestamp('2024-09-01 00:00:00')

In [19]:
# Ensure date column is in datetime format
df1['JOB_ENDDATE'] = pd.to_datetime(df1['JOB_ENDDATE'])

KeyError: 'JOB_ENDDATE'

In [ ]:
# Ensure the index is unique
df = df[~df.index.duplicated(keep='first')]


In [17]:
# Set date column as index
df1.set_index('JOB_ENDDATE', inplace=True)

In [18]:
# Split data into training and testing sets
train = df1[:pd.Timestamp('2024-08-31')]
test = df1[pd.Timestamp('2024-09-01'):]

KeyError: "Cannot get right slice bound for non-unique label: Timestamp('2024-08-31 00:00:00')"

In [ ]:
# Split data into training and testing sets
train = df1[:'2024-08']
test = df1['2024-09':]

In [ ]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [ ]:
monthly_data